In [ ]:
# Load Dependencies

import os.path as op
import os
from IPython.display import clear_output
import pickle
import mne
import pyxdf
import pandas as pd
import seaborn as sns
from utils import Preprocessing

In [ ]:
# Specify range for participant pairs
# Give zero padding to keep BIDS compliant
subjs = ["%03d" % x for x in range(0,182)]

In [ ]:
# Specify the number of sessions
# Max for any given participant pair is 3
# Some participant pairs may have < 3 sessions due to technical error
sessions = ["%03d" % x for x in range(1,4)]

In [ ]:
# Specify top-level directory where data are stored
# You will need to update this based on where you saved the data
prefix = "/mnt/ccsl_nas/box_sync/tangram_eeg/study1/raw_data/"

In [ ]:
# Make file path convention
def make_file_path(prefix, subj, session):
    return prefix + "sub-P"+subj+"/ses-S"+session+"/eeg/sub-P"+subj+"_ses-S"+session+"_task-Default_run-001_eeg.xdf"

In [ ]:
# Import participant data and run autoreject
# NOTE: Epoch duration is specified in line 205 of Preprocessing.py
# Set `duration=##` to whatever value you want the epoch length to be, in seconds

# preprocess_dict is where synchronized streams + autoreject for each participant pair for each session are stored
preprocess_dict = {}

for subj in subjs:
    subj_df_dict = {}
    for session in sessions:
        
        # Reference each xdf file for each pair of participants for each session
        file = make_file_path(prefix, subj, session)
        
        # Read in xdf file, synchronize streams, and structure for each participant for each electrode
        if os.path.isfile(file):
            try:
                print("processing subj: {} session: {}".format(subj, session))
                streams, header  = pyxdf.load_xdf(file, select_streams=[{'type': 'EEG'}])
                streams = streams[:2]
                streams = Preprocessing._sync_timestamps(streams, kind='linear')
                streams = Preprocessing._limit_streams_to_overlap(streams)
                
                if streams[0]['info']['name'][0][5:9] == 'AC76':
                    director = streams[0]
                    matcher = streams[1]
                elif streams[0]['info']['name'][0][5:9] == 'AC7D':
                    director = streams[1]
                    matcher = streams[0]
                else:
                    raise Exception("stream naming incorrect")
                
                # Timeseries for first participant in the xdf
                # Runs autoreject (from preprocessing.py)
                # Autoreject parameters are specified in Preprocssing.py
                timeseries_p1 = Preprocessing.preprocessing_autoreject(director)
                # Timeseries for second participant in the xdf
                # Runs autoreject (from preprocessing.py)
                # Autoreject parameters are specified in Preprocssing.py
                timeseries_p2 = Preprocessing.preprocessing_autoreject(matcher)
                
                # Structure autorejected timeseries for both participants
                pair_df = pd.concat([timeseries_p1, timeseries_p2.iloc[:,3:]], axis=1)
                
                # Name column headers
                pair_df.columns = ["index", "epoch", "time", "TP9-1", "AF7-1", "AF8-1", "TP10-1", "TP9-2", "AF7-2", "AF8-2", "TP10-2"]
                clear_output()

                subj_df_dict[session] = pair_df
            except:
                print("Failed subj: {} session: {}".format(subj, session))
            
    if os.path.exists(prefix+"sub-P"+subj):
        preprocess_dict[subj] = subj_df_dict

In [ ]:
# Filehandler is for writing a pickle file
# Update filename based on epoch duration
filehandler = open("preprocessed_interpolate_bandpass_.1_20/preprocessed_data_.5s_epoch.obj","wb")

# Pickle preprocess_dict
pickle.dump(preprocess_dict,filehandler)
filehandler.close()